In [29]:
import joblib
import pandas as pd
import torch
import torch.nn as nn
from transformers import RobertaTokenizer, RobertaForSequenceClassification
import numpy as np

In [1]:
!pip install joblib

In [30]:
df = pd.read_csv('bitext_free_dataset.csv')
df.head(3)

,flags,utterance,category,intent
0,BILC,"I don't have an online account, what do I have...",ACCOUNT,create_account
1,BILQZ,can you tell me if i can regisger two accounts...,ACCOUNT,create_account
2,BPLC,"I have no online account, open one, please",ACCOUNT,create_account


In [32]:
df['category'].unique()

array(['ACCOUNT', 'CANCELLATION_FEE', 'CONTACT', 'DELIVERY', 'FEEDBACK',
       'INVOICES', 'NEWSLETTER', 'ORDER', 'PAYMENT', 'REFUNDS',
       'SHIPPING'], dtype=object)

In [18]:
df.loc[10000,:]

flags                                              BMLDW
utterance    ask an agent where to check my fucking bill
category                                        INVOICES
intent                                    check_invoices
Name: 10000, dtype: object

In [7]:
model = RobertaForSequenceClassification.from_pretrained('model')
tokenizer = RobertaTokenizer.from_pretrained('model')

In [19]:
text = "ask an agent where to check my fucking bill"
inputs = tokenizer(text, return_tensors="pt")
outputs = model(**inputs)

In [20]:
predictions = []

with torch.no_grad():
  logits = outputs.logits
  predictions.extend(torch.sigmoid(logits).cpu().numpy())
  predictions = (np.array(predictions) >= 0.5).astype(int)

In [21]:
predictions

array([[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]])

In [22]:
loaded_encoder = joblib.load('model/onehot_encoder.joblib')

In [23]:
loaded_encoder.inverse_transform(predictions)

array([['INVOICES']], dtype=object)